In [1]:
import pandas as pd
import time

In [2]:
train_data = pd.read_csv("../datasets/train_preprocess.csv")

In [3]:
train_data.head(1)

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,Year,Month,Day,...,PdDistrictID,HasBlock,RoadType,RoadTypeID,RoadName1,RoadName2,RoadBlock,RoadBlockID,RoadName1ID,RoadName2ID
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,...,4,0,CrossRoad,5,OAK,LAGUNA,Unkown,84,1296,842


In [4]:
feature_names = ['Year', 'Month', 'Hour', 'DayOfWeekID', 'PdDistrictID', \
                 'HasBlock', 'RoadTypeID', 'RoadBlockID', 'RoadName1ID', 'RoadName2ID', 'X', 'Y']

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [6]:
# n_estimators=’warn’, criterion=’gini’, max_depth=None, 
# min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
# max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
# min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, 
# random_state=None, verbose=0, warm_start=False, class_weight=None

In [7]:
rfclf = RandomForestClassifier()

In [8]:
X = train_data[feature_names]
y_true = train_data["Category"]

In [9]:
# start = time.time()
# rfclf.fit(X, y_true)
# print("Training with total dataset needs %d seconds." % (time.time()-start))

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import log_loss, make_scorer, auc

In [11]:
param_grid = { "n_estimators": list(range(200, 1500, 200)),
#              "criterion": ["gini", "entropy"],
#              "max_depth": list(range(start=6, end=11, step=1)),
#              "min_samples_split": list(range(start=20, end=51, step=10)),
#              "max_features": list(range(start=0.5, end=1.1, step=0.1)),
#              "bootstrap": [True, False],
             }

cv_clf_trees = GridSearchCV(estimator=rfclf, param_grid=param_grid, scoring=make_scorer(log_loss), 
                      n_jobs=2, cv=10, verbose=3, return_train_score=True)

start = time.time()
cv_clf_trees.fit(X, y_true)
print("Training with total dataset needs %d seconds." % (time.time()-start))

Fitting 10 folds for each of 7 candidates, totalling 70 fits


d:\Anaconda3\envs\gym\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


MemoryError: could not allocate 327155712 bytes